In [3]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [4]:
def get_data(path, noise=False):
    data = np.load(path)
    if noise == True :
        data = data + np.random.normal(0, 0.001, data.shape)
    return data

In [5]:
X_train = get_data('enron-train-features.pkl').astype(np.uint8)
y_train = get_data('enron-train-labels.pkl').astype(np.int8)
X_test = get_data('enron-test-features.pkl').astype(np.uint8)
y_test = get_data('enron-test-labels.pkl').astype(np.int8)

In [6]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (1123, 1001)
Train_Y:  (1123, 53)
Test_X:  (579, 1001)
Test_Y:  (579, 53)


In [7]:
def hamming_accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

## Chain Model type 1


In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.backend.tensorflow_backend import clear_session

Using TensorFlow backend.


In [9]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform',name='first'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform',name='nretrain1'))
    model.add(Dropout(0.4))
    model.add(Dense(128,activation='relu',kernel_initializer='glorot_uniform',name='nretrain2'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform',name='last'))
    model.add(Dropout(0.4))    
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform',name='output'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [10]:
#for calculating time
from time import time

In [12]:
#Model chain
chain = []

#Training
X_feed = X_train.copy()
t1 = time()
for i in range(y_train.shape[1]):
    print("Training chain node ",i)
    y_now = y_train[:,[i,]].copy()
    print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

    node = create_c1node(X_feed,y_now)
    node.fit(X_feed,y_now,epochs=10,batch_size=50)
    print("Training of node {} complete\n\n".format(i))  
    #Checking accuracy of the node
    #---
    #Adding node to chain 
    chain.append(node)
    #Updating X_feed
    X_feed = np.append(X_feed,y_now,axis=1)
t2 = time()
print("Time taken: ",(t2-t1))

Training chain node  0
Shapes:
 X = (1123, 1001) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [==============================] - 1s 946us/step - loss: 0.3529 - acc: 0.8664
Epoch 2/10
1123/1123 [==============================] - 0s 266us/step - loss: 0.1549 - acc: 0.9840
Epoch 3/10
1123/1123 [==============================] - 0s 260us/step - loss: 0.1169 - acc: 0.9840
Epoch 4/10
1123/1123 [==============================] - 0s 257us/step - loss: 0.0999 - acc: 0.9840
Epoch 5/10
1123/1123 [==============================] - 0s 279us/step - loss: 0.0926 - acc: 0.9840
Epoch 6/10
1123/1123 [==============================] - 0s 313us/step - loss: 0.0819 - acc: 0.9840
Epoch 7/10
1123/1123 [==============================] - 0s 284us/step - loss: 0.0772 - acc: 0.9840
Epoch 8/10
1123/1123 [==============================] - 0s 256us/step - loss: 0.0674 - acc: 0.9840
Epoch 9/10
1123/1123 [==============================] - ETA: 0s - loss: 0.0630 - acc: 0.982 - 0s 299us/step - loss: 0.0590 - acc: 0.9840
Epoch 

1123/1123 [==============================] - 0s 274us/step - loss: 0.0579 - acc: 0.9751
Epoch 7/10
1123/1123 [==============================] - 0s 257us/step - loss: 0.0458 - acc: 0.9786
Epoch 8/10
1123/1123 [==============================] - 0s 254us/step - loss: 0.0424 - acc: 0.9884
Epoch 9/10
1123/1123 [==============================] - 0s 289us/step - loss: 0.0336 - acc: 0.9911
Epoch 10/10
1123/1123 [==============================] - 0s 258us/step - loss: 0.0324 - acc: 0.9938
Training of node 7 complete


Training chain node  8
Shapes:
 X = (1123, 1009) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [==============================] - 2s 1ms/step - loss: 0.2167 - acc: 0.9457
Epoch 2/10
1123/1123 [==============================] - 0s 261us/step - loss: 0.0944 - acc: 0.9938
Epoch 3/10
1123/1123 [==============================] - 0s 272us/step - loss: 0.0861 - acc: 0.9938
Epoch 4/10
1123/1123 [==============================] - 0s 272us/step - loss: 0.0763 - acc: 0.9938
Epoch 5/10
1123/1123 [====

Epoch 1/10
1123/1123 [==============================] - 2s 2ms/step - loss: 0.1845 - acc: 0.9617
Epoch 2/10
1123/1123 [==============================] - 0s 273us/step - loss: 0.0637 - acc: 0.9955
Epoch 3/10
1123/1123 [==============================] - 0s 284us/step - loss: 0.0717 - acc: 0.9955
Epoch 4/10
1123/1123 [==============================] - 0s 298us/step - loss: 0.0586 - acc: 0.9955
Epoch 5/10
1123/1123 [==============================] - 0s 294us/step - loss: 0.0618 - acc: 0.9955
Epoch 6/10
1123/1123 [==============================] - 0s 304us/step - loss: 0.0507 - acc: 0.9955
Epoch 7/10
1123/1123 [==============================] - 0s 286us/step - loss: 0.0486 - acc: 0.9955
Epoch 8/10
1123/1123 [==============================] - 0s 305us/step - loss: 0.0489 - acc: 0.9955TA: 0s - loss: 0.0567 - acc: 0.9
Epoch 9/10
1123/1123 [==============================] - 0s 311us/step - loss: 0.0473 - acc: 0.9955
Epoch 10/10
1123/1123 [==============================] - 0s 293us/step - loss: 

1123/1123 [==============================] - 0s 298us/step - loss: 0.0452 - acc: 0.9653
Epoch 7/10
1123/1123 [==============================] - 0s 286us/step - loss: 0.0383 - acc: 0.9688
Epoch 8/10
1123/1123 [==============================] - 0s 290us/step - loss: 0.0308 - acc: 0.9822
Epoch 9/10
1123/1123 [==============================] - 0s 289us/step - loss: 0.0254 - acc: 0.9911
Epoch 10/10
1123/1123 [==============================] - 0s 304us/step - loss: 0.0204 - acc: 0.9938
Training of node 22 complete


Training chain node  23
Shapes:
 X = (1123, 1024) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [==============================] - 3s 2ms/step - loss: 0.3538 - acc: 0.8985
Epoch 2/10
1123/1123 [==============================] - 0s 337us/step - loss: 0.2184 - acc: 0.9377
Epoch 3/10
1123/1123 [==============================] - 0s 300us/step - loss: 0.1669 - acc: 0.9377
Epoch 4/10
1123/1123 [==============================] - 0s 303us/step - loss: 0.1194 - acc: 0.9377
Epoch 5/10
1123/1123 [==

Epoch 1/10
1123/1123 [==============================] - 3s 3ms/step - loss: 0.1606 - acc: 0.9519
Epoch 2/10
1123/1123 [==============================] - 0s 322us/step - loss: 0.0296 - acc: 0.9982
Epoch 3/10
1123/1123 [==============================] - 0s 319us/step - loss: 0.0289 - acc: 0.9982
Epoch 4/10
1123/1123 [==============================] - 0s 314us/step - loss: 0.0288 - acc: 0.9982
Epoch 5/10
1123/1123 [==============================] - 0s 301us/step - loss: 0.0288 - acc: 0.9982
Epoch 6/10
1123/1123 [==============================] - 0s 297us/step - loss: 0.0288 - acc: 0.9982
Epoch 7/10
1123/1123 [==============================] - 0s 297us/step - loss: 0.0278 - acc: 0.9982
Epoch 8/10
1123/1123 [==============================] - 0s 321us/step - loss: 0.0253 - acc: 0.9982
Epoch 9/10
1123/1123 [==============================] - 0s 294us/step - loss: 0.0194 - acc: 0.9982
Epoch 10/10
1123/1123 [==============================] - 0s 296us/step - loss: 0.0192 - acc: 0.9982
Training of

Epoch 7/10
1123/1123 [==============================] - 0s 303us/step - loss: 0.0242 - acc: 0.9849
Epoch 8/10
1123/1123 [==============================] - 0s 303us/step - loss: 0.0164 - acc: 0.9938
Epoch 9/10
1123/1123 [==============================] - 0s 307us/step - loss: 0.0108 - acc: 0.9973
Epoch 10/10
1123/1123 [==============================] - 0s 305us/step - loss: 0.0160 - acc: 0.9955
Training of node 37 complete


Training chain node  38
Shapes:
 X = (1123, 1039) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [==============================] - 4s 3ms/step - loss: 0.2319 - acc: 0.9386
Epoch 2/10
1123/1123 [==============================] - 0s 303us/step - loss: 0.0959 - acc: 0.9929
Epoch 3/10
1123/1123 [==============================] - 0s 302us/step - loss: 0.0771 - acc: 0.9929
Epoch 4/10
1123/1123 [==============================] - 0s 303us/step - loss: 0.0608 - acc: 0.9929
Epoch 5/10
1123/1123 [==============================] - 0s 306us/step - loss: 0.0547 - acc: 0.9929
Epoch 6/10
11

Epoch 2/10
1123/1123 [==============================] - 0s 319us/step - loss: 3.5871e-04 - acc: 1.0000
Epoch 3/10
1123/1123 [==============================] - 0s 310us/step - loss: 1.3090e-04 - acc: 1.0000
Epoch 4/10
1123/1123 [==============================] - 0s 310us/step - loss: 7.9603e-05 - acc: 1.0000
Epoch 5/10
1123/1123 [==============================] - 0s 311us/step - loss: 4.3560e-05 - acc: 1.0000
Epoch 6/10
1123/1123 [==============================] - 0s 312us/step - loss: 2.9989e-05 - acc: 1.0000
Epoch 7/10
1123/1123 [==============================] - 0s 309us/step - loss: 5.3366e-05 - acc: 1.0000
Epoch 8/10
1123/1123 [==============================] - 0s 313us/step - loss: 3.4408e-05 - acc: 1.0000
Epoch 9/10
1123/1123 [==============================] - 0s 310us/step - loss: 1.6697e-05 - acc: 1.0000
Epoch 10/10
1123/1123 [==============================] - 0s 313us/step - loss: 8.1049e-05 - acc: 1.0000
Training of node 45 complete


Training chain node  46
Shapes:
 X = (112

1123/1123 [==============================] - 0s 325us/step - loss: 0.0144 - acc: 0.9991
Epoch 8/10
1123/1123 [==============================] - 0s 318us/step - loss: 0.0144 - acc: 0.9991
Epoch 9/10
1123/1123 [==============================] - 0s 328us/step - loss: 0.0144 - acc: 0.9991
Epoch 10/10
1123/1123 [==============================] - 0s 318us/step - loss: 0.0144 - acc: 0.9991
Training of node 52 complete


Time taken:  357.972017288208


In [9]:
#Model chain
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print("Getting op from chain node ",i)
    node_name = "Node" + str(i)
    print("Shapes:\n X = {}".format(X_feed.shape))
    output = node.predict(X_feed)
    output = output.round().astype(int)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i+=1
    #Updating X_feed
            

Getting op from chain node  0
Shapes:
 X = (579, 1001)
Getting op from chain node  1
Shapes:
 X = (579, 1002)
Getting op from chain node  2
Shapes:
 X = (579, 1003)
Getting op from chain node  3
Shapes:
 X = (579, 1004)
Getting op from chain node  4
Shapes:
 X = (579, 1005)
Getting op from chain node  5
Shapes:
 X = (579, 1006)
Getting op from chain node  6
Shapes:
 X = (579, 1007)
Getting op from chain node  7
Shapes:
 X = (579, 1008)
Getting op from chain node  8
Shapes:
 X = (579, 1009)
Getting op from chain node  9
Shapes:
 X = (579, 1010)
Getting op from chain node  10
Shapes:
 X = (579, 1011)
Getting op from chain node  11
Shapes:
 X = (579, 1012)
Getting op from chain node  12
Shapes:
 X = (579, 1013)
Getting op from chain node  13
Shapes:
 X = (579, 1014)
Getting op from chain node  14
Shapes:
 X = (579, 1015)
Getting op from chain node  15
Shapes:
 X = (579, 1016)
Getting op from chain node  16
Shapes:
 X = (579, 1017)
Getting op from chain node  17
Shapes:
 X = (579, 1018)
Ge

In [10]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [11]:
# accuracy
from sklearn.metrics import hamming_loss, log_loss, f1_score,accuracy_score
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("Hamming accuracy = ",hamming_accuracy_score(y_test,predictions))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions))
print("F1 score = ",f1_score(y_test,predictions,average='macro'))

Accuracy =  0.155440414508
Hamming accuracy =  0.448512009004
Hamming loss =  0.05122690390067455
Log loss =  60.5806239444
F1 score =  0.20066038712
